<a href="https://colab.research.google.com/github/Rosangelafl/exercicios_python_inovacao_civica/blob/main/desafio2_m5_raspagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Licitações do DOU - Diário oficial da união

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install requests-html

     |████████████████████████████████| 83 kB 2.0 MB/s 
     |████████████████████████████████| 138 kB 60.5 MB/s 
     |████████████████████████████████| 103 kB 55.5 MB/s 
     |████████████████████████████████| 127 kB 46.2 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=b1e2454f85446e18fe5001136d3768dc5b9d1ac82cae6f4487552234a824c362
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=0a583de4824ca73d99d1443bf83f5123bda236ce180e4a5540eeebca032603a5
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [2]:
!pip install folium==0.2.1

     |████████████████████████████████| 69 kB 5.4 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79809 sha256=fe950c92e23ff1f46a7e7acbc61a88020739bf99cb9d71e893d35648d202dece
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [3]:
import datetime
import pandas as pd
from requests_html import HTMLSession

In [4]:
url = "http://comprasnet.gov.br/ConsultaLicitacoes/ConsLicitacaoDia.asp"
session = HTMLSession()
response = session.get(url)

In [6]:
licitacoes = []
rows = response.html.xpath("//form")
for row in rows:
    descricao = row.xpath("./form/table/tr[2]//text()")
    descricao = '\n'.join(descricao).strip()
    item = {
        "data": datetime.date.today(),
        "descricao": descricao,
    }
    licitacoes.append(item)

#### Adicionando ao DataFrame

In [7]:
df_licitacoes = pd.DataFrame(licitacoes)

Replicando para todas as páginas.

Recomenda-se transformar a extração de conteúdo da página em uma função que pode ser chamada para cada página.

In [8]:
def extract_licitacoes(response):
    """Se desejar, preencha esta função"""
    licitacoes = []
    rows = response.html.xpath("//form")
    for row in rows:
        descricao = row.xpath("./form/table/tr[2]//text()")
        descricao = '\n'.join(descricao).strip()
        item = {
            "data": datetime.date.today(),
            "descricao": descricao,
        }
        licitacoes.append(item)
    return licitacoes

In [9]:
page = 1
next_page_button = response.html.xpath("//input[@name='proxima']")
while next_page_button:
    page += 1
    next_page_url = f"{url}?pagina={page}"
    response = session.get(next_page_url)
    licitacoes = extract_licitacoes(response)
    df_licitacoes = df_licitacoes.append(pd.DataFrame(licitacoes))
    next_page_button = response.html.xpath("//input[@name='proxima']")

Salvando em JSON

In [10]:
df_licitacoes.to_json('licitacoes.json', orient="records", date_format="iso")